In [1]:
from keras.models import model_from_json
from keras.models import load_model
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import backend as K
#K.set_image_dim_ordering('th')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
import pandas as pd
import csv
from os.path import splitext
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from tensorflow.keras.callbacks import TensorBoard

Using TensorFlow backend.


# Load training and test data into dataframes

In [7]:

PATH = os.getcwd()
from keras.utils import to_categorical
df = pd.read_csv(PATH + '\\CNN-image-dataset.csv');
data_test = df[5001:]

data_test = data_test.iloc[np.random.permutation(len(data_test))]

In [9]:
testdf = data_test[10:20]

import pandas as pd
from pandas.table.plotting import table # EDIT: see deprecation warnings below

ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, testdf)  # where df is your data frame

plt.savefig('mytable.png')

ModuleNotFoundError: No module named 'pandas.table'

# Prepering CNN Model Test Data

In [3]:


img_rows, img_cols = 28, 28

# X_test forms the test images, and y_test forms the test labels
X_test = np.array(data_test.iloc[:, 3:])
y_test = to_categorical(np.array(data_test.iloc[:, 1]))


# Prepare the test images
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_test = X_test.astype('float32')
X_test /= 255


# load json and create model

In [4]:
# load RNN Model
json_file = open('cnn4.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("cnn4.h5")
print("Loaded model from disk")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


In [ ]:
#preducting test data probability
CNN_pred = loaded_model.predict(X_test)

# Prepering RNN Model Test Data

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 250
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data_test['productDisplayName'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


X_test_rnn = np.array(data_test.iloc[:, 2])
X_rnn = tokenizer.texts_to_sequences(X_test_rnn)
X_rnn = pad_sequences(X_rnn, maxlen=MAX_SEQUENCE_LENGTH)

Found 1139 unique tokens.


# Loadin RNN Model

In [6]:
# load RNN Model
json_file = open('rnnmodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
RNN_loaded_model = model_from_json(loaded_model_json)
# load weights into new model
RNN_loaded_model.load_weights("rnnmodel.h5")
print("Loaded model from disk")

Loaded model from disk


In [7]:
#preducting test data probability
RNN_pred = RNN_loaded_model.predict_proba(X_rnn)

In [8]:
RNN_loaded_model.predict_proba(X_rnn)
# get the predictions for the test data
predicted_classes = RNN_loaded_model.predict_classes(X_rnn)
print(len(predicted_classes))
# get the indices to be plotted

y_true = data_test.iloc[:, 1].values
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]
print("Correct Data classified:",len(correct))
print("Incorrect Data classified:",len(incorrect))

5000
Correct Data classified: 3933
Incorrect Data classified: 1067


In [9]:
#Multipying the CNN - RNN probabilities
final_pred_prob = np.multiply(CNN_pred, RNN_pred)

In [10]:
labels = [0, 1, 2]
finalPred_class = []
for pred in final_pred_prob:
    finalPred_class.append(labels[np.argmax(pred)])

In [11]:
# get the predictions for the test data
predicted_classes = finalPred_class
print(len(predicted_classes))
# get the indices to be plotted

y_true = data_test.iloc[:, 1]
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]
print(len(correct))
print(len(incorrect))

5000
4992
8


C:\Users\rohit\Miniconda3\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


In [12]:
RNN_loaded_model.predict_proba(X_rnn)

array([[9.99935389e-01, 5.47545824e-05, 9.93227695e-06],
       [9.99930859e-01, 3.87936598e-05, 3.03900797e-05],
       [9.89893973e-01, 9.71133728e-03, 3.94765899e-04],
       ...,
       [9.99992251e-01, 1.82209283e-06, 5.99874556e-06],
       [9.99995828e-01, 1.69293378e-06, 2.44586568e-06],
       [9.99984264e-01, 3.86559623e-06, 1.18878015e-05]], dtype=float32)

In [13]:
loaded_model.predict_proba(X_test)

array([[9.9985957e-01, 4.3435910e-05, 9.7050375e-05],
       [9.9987566e-01, 3.9093826e-05, 8.5226537e-05],
       [9.9986362e-01, 3.0027961e-05, 1.0627679e-04],
       ...,
       [9.9986100e-01, 2.6102463e-05, 1.1282336e-04],
       [9.9990916e-01, 2.1855969e-05, 6.9017704e-05],
       [9.9976486e-01, 4.8791790e-05, 1.8638598e-04]], dtype=float32)